In [428]:
#pip install gluonts

In [429]:
#pip install huggingface_hub

In [430]:
#pip install -e '.[notebook]'


In [431]:
#%pip install huggingface_hub

In [432]:
#!git clone https://github.com/SalesforceAIResearch/uni2ts.git

In [433]:
#%pip install -e '.[notebook]'

In [434]:
#%pip install --upgrade torch torchvision torchaudio pytorch-lightning

In [435]:
!git clone https://github.com/SalesforceAIResearch/uni2ts.git

fatal: destination path 'uni2ts' already exists and is not an empty directory.


In [436]:
!cd uni2ts

In [437]:
import os
os.chdir("uni2ts")  # Move into the cloned folder

In [438]:
from uni2ts.model.moirai import MoiraiForecast

In [439]:
from uni2ts.model.moirai import MoiraiModule

In [440]:
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.util import to_pandas
from gluonts.evaluation import make_evaluation_predictions

In [441]:
from gluonts.dataset.split import split

In [442]:
from huggingface_hub import hf_hub_download

In [443]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Use only GPU

In [444]:
import torch
torch.set_float32_matmul_precision('medium')

In [ ]:
data= pd.read_csv("data/final_concatenated_data.csv")
data

,Unnamed: 0,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,1406,8/22/2024,297.72,294.23,0.17,0.17,30.02,559.39,0.0,358,...,1.72,7.289,0.050,7.466,2.75,2.75,6.13,2.85,2.78,3.88
1407,1407,8/23/2024,240.47,293.74,0.45,0.45,0.29,491.92,0.0,375,...,0.04,7.360,0.000,7.447,2.75,2.75,6.18,2.90,2.81,3.88
1408,1408,8/24/2024,242.45,293.75,0.00,0.00,0.00,439.40,0.0,377,...,0.00,7.360,0.010,7.435,2.75,2.75,6.20,2.88,2.80,3.88
1409,1409,8/25/2024,286.80,294.24,0.00,0.00,0.00,470.49,0.0,380,...,0.00,7.345,0.270,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [446]:
# Convert 'date' column to proper datetime format
data['date'] = pd.to_datetime(data['date'])

# Verify the column types to confirm the 'date' is now datetime
print(data.info())

# When displaying or exporting, you can format the date as MM/DD/YYYY
data['formatted_date'] = data['date'].dt.strftime('%m/%d/%Y')

# Print the first few rows to see the formatted date
print(data[['date', 'formatted_date']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1411 entries, 0 to 1410
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Unnamed: 0        1411 non-null   int64         
 1   date              1411 non-null   datetime64[ns]
 2   S199              1411 non-null   float64       
 3   S200              1411 non-null   float64       
 4   S332B             1411 non-null   float64       
 5   S332BN            1411 non-null   float64       
 6   S332C             1411 non-null   float64       
 7   S332D             1411 non-null   float64       
 8   S332DX1           1411 non-null   float64       
 9   S12A              1411 non-null   int64         
 10  S12B              1411 non-null   int64         
 11  S12C              1411 non-null   int64         
 12  S12D              1411 non-null   int64         
 13  S333              1411 non-null   float64       
 14  S333N             1411 n

In [447]:
# Drop the 'formatted_date' column if not needed
data = data.drop(columns=['Unnamed: 0','formatted_date'])

data['date'] = data['date'].dt.strftime('%m/%d/%Y')

# Check the data to confirm the index
data

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,NP205_rain,NP205_stage,P33_rain,P33_stage,NP205_PET,P33_PET,G620_water_level,SWEVER4_stage,TSH_stage,NP62_stage
0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,382,...,2.28,6.925,0.106,7.133,2.09,2.09,5.92,2.92,3.03,4.04
1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,405,...,0.01,7.025,0.000,7.133,3.87,3.87,5.97,2.95,3.08,4.05
2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,394,...,0.00,7.008,0.088,7.138,3.51,3.51,5.98,2.93,3.06,4.05
3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,392,...,0.11,6.986,0.256,7.167,1.98,1.98,5.98,2.91,3.05,4.06
4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,399,...,0.37,6.977,0.678,7.195,1.27,1.27,5.99,2.90,3.06,4.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,08/22/2024,297.72,294.23,0.17,0.17,30.02,559.39,0.0,358,250,...,1.72,7.289,0.050,7.466,2.75,2.75,6.13,2.85,2.78,3.88
1407,08/23/2024,240.47,293.74,0.45,0.45,0.29,491.92,0.0,375,265,...,0.04,7.360,0.000,7.447,2.75,2.75,6.18,2.90,2.81,3.88
1408,08/24/2024,242.45,293.75,0.00,0.00,0.00,439.40,0.0,377,268,...,0.00,7.360,0.010,7.435,2.75,2.75,6.20,2.88,2.80,3.88
1409,08/25/2024,286.80,294.24,0.00,0.00,0.00,470.49,0.0,380,272,...,0.00,7.345,0.270,7.440,2.75,2.75,6.19,2.86,2.80,3.98


In [448]:
# Replace with your actual DataFrame
df = pd.DataFrame(data)

In [449]:
#Convert 'date' to datetime format if not 
df['date'] = pd.to_datetime(df['date'])

#Melt the DataFrame to long format
long_df = pd.melt(df, id_vars=['date'], 
                  value_vars=[col for col in df.columns if col != 'date'], 
                  var_name='unique_id', 
                  value_name='y')

#Rename the date column to 'ds'
long_df.rename(columns={'date': 'ds'}, inplace=True)

# Display the resulting long format DataFrame
print(long_df)

              ds   unique_id       y
0     2020-10-16        S199  242.17
1     2020-10-17        S199  296.05
2     2020-10-18        S199  277.85
3     2020-10-19        S199  224.13
4     2020-10-20        S199  260.51
...          ...         ...     ...
52202 2024-08-22  NP62_stage    3.88
52203 2024-08-23  NP62_stage    3.88
52204 2024-08-24  NP62_stage    3.88
52205 2024-08-25  NP62_stage    3.98
52206 2024-08-26  NP62_stage    4.03

[52207 rows x 3 columns]


In [450]:
long_df['ds'] = pd.to_datetime(long_df['ds'])
# Set 'ds' as the index
long_df.set_index('ds', inplace=True)
long_df

,unique_id,y
ds,,
2020-10-16,S199,242.17
2020-10-17,S199,296.05
2020-10-18,S199,277.85
2020-10-19,S199,224.13
2020-10-20,S199,260.51
...,...,...
2024-08-22,NP62_stage,3.88
2024-08-23,NP62_stage,3.88
2024-08-24,NP62_stage,3.88


In [451]:
### Creating Y_df as separate without ds as index, for getting Y_test_df 
# Resetting the index and keeping the 'ds' column as a regular column
Y_df = long_df.reset_index()
Y_df

,ds,unique_id,y
0,2020-10-16,S199,242.17
1,2020-10-17,S199,296.05
2,2020-10-18,S199,277.85
3,2020-10-19,S199,224.13
4,2020-10-20,S199,260.51
...,...,...,...
52202,2024-08-22,NP62_stage,3.88
52203,2024-08-23,NP62_stage,3.88
52204,2024-08-24,NP62_stage,3.88
52205,2024-08-25,NP62_stage,3.98


In [452]:
# Count occurrences of each unique ID
unique_id_counts = Y_df['unique_id'].value_counts()

# Display the result
print(unique_id_counts)

unique_id
S199                1411
S200                1411
S332B               1411
S332BN              1411
S332C               1411
S332D               1411
S332DX1             1411
S12A                1411
S12B                1411
S12C                1411
S12D                1411
S333                1411
S333N               1411
S334                1411
S343A               1411
S343B               1411
S344                1411
S356                1411
S18C_S              1411
S12A_T              1411
S12C_T              1411
S12D_T              1411
S12B_T              1411
S333_T              1411
S334_H              1411
NESRS1              1411
NESRS2              1411
NP205_rain          1411
NP205_stage         1411
P33_rain            1411
P33_stage           1411
NP205_PET           1411
P33_PET             1411
G620_water_level    1411
SWEVER4_stage       1411
TSH_stage           1411
NP62_stage          1411
Name: count, dtype: int64


In [453]:
n_time = len(Y_df.ds.unique())
val_size = int(0.15 * n_time)
test_size = int(0.15 * n_time)

In [454]:
print('n_time=', n_time)
print('val_size', val_size)
print('test_size', test_size)

n_time= 1411
val_size 211
test_size 211


In [455]:
#For the .fit method, model to be trained on provided dataset
Y_train_df = Y_df[Y_df['ds'] < Y_df['ds'].values[-422]]  # Training set
Y_val_df = Y_df[(Y_df['ds'] >= Y_df['ds'].values[-422]) & (Y_df['ds'] < Y_df['ds'].values[-211])]  # Validation set
Y_test_df = Y_df[Y_df['ds'] >= Y_df['ds'].values[-211]].reset_index(drop=True)

In [456]:
Y_test_df

,ds,unique_id,y
0,2024-01-29,S199,77.29
1,2024-01-30,S199,76.12
2,2024-01-31,S199,76.11
3,2024-02-01,S199,76.04
4,2024-02-02,S199,76.01
...,...,...,...
7802,2024-08-22,NP62_stage,3.88
7803,2024-08-23,NP62_stage,3.88
7804,2024-08-24,NP62_stage,3.88
7805,2024-08-25,NP62_stage,3.98


In [457]:
# Create GluonTS dataset and specify the frequency
ds = PandasDataset.from_long_dataframe(long_df, target="y", item_id="unique_id", freq="D")   

In [458]:
# Group time series into a multivariate dataset (adjust the group size accordingly)
grouper = MultivariateGrouper(len(ds))
multivar_ds = grouper(ds)

In [459]:
# Split into train/test set (specify TEST value)
#TEST = 211  # define TEST as per your requirement
#train, test_template = split(
    #multivar_ds, offset=-TEST
#)  # assign last TEST time steps as test set

In [460]:
# Split into train/test set (specify TEST value)
TEST = 211  # define TEST as per your requirement
train, test_template = split(
    multivar_ds, offset=-TEST
)  # assign last TEST time steps as test set

In [461]:
#setup rolling window
PDT = 28  # specify prediction length
test_data = test_template.generate_instances(
    prediction_length=PDT,
    windows=TEST // PDT,
    distance=1,  # distance between each window
)

In [462]:
SIZE = "large"  # model size: choose from {'small', 'base', 'large'}
CTX = 100  # context length: any positive integer
PSZ = 16 
BSZ = 32  # batch size: any positive integer

In [463]:
# Prepare and load model (check your specific model setup for sizes)
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,  # set context length appropriately
    patch_size=PSZ,  # set patch size
    num_samples=211,
    target_dim=len(ds),
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real
)

In [464]:
# Set up the predictor
predictor = model.create_predictor(batch_size=32)

In [465]:
# Get forecasts (this will output the forecasted values)
forecasts = predictor.predict(test_data.input)

In [466]:
forecasts

<generator object PyTorchPredictor.predict at 0x7fc78bdcfae0>

In [467]:
# Generate predictions and comparison
input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

In [468]:
inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

In [469]:
# Display the first input, label, and forecast
print("Input:", inp)
print("Label:", label)
print("Forecast:", forecast)

Input: {'target': array([[  5.92,   5.97,   5.98, ...,   6.02,   6.01,   6.  ],
       [  7.75,   7.85,   7.89, ...,   8.01,   8.01,   8.  ],
       [  7.94,   7.96,   7.98, ...,   8.  ,   8.  ,   8.  ],
       ...,
       [451.26, 531.02, 454.09, ..., 393.01, 393.95, 394.68],
       [  2.92,   2.95,   2.93, ...,   2.75,   2.74,   2.72],
       [  3.03,   3.08,   3.06, ...,   2.75,   2.73,   2.71]],
      dtype=float32), 'start': Period('2020-10-16', 'D'), 'feat_static_cat': array([0], dtype=int32)}
Label: {'target': array([[  5.99,   5.97,   5.96, ...,   6.01,   6.01,   5.99],
       [  7.99,   7.98,   7.97, ...,   8.06,   8.05,   8.02],
       [  7.99,   7.98,   7.98, ...,   8.02,   8.02,   8.  ],
       ...,
       [394.49, 393.54, 393.81, ..., 394.08, 394.07, 393.5 ],
       [  2.7 ,   2.68,   2.66, ...,   2.55,   2.54,   2.52],
       [  2.7 ,   2.69,   2.67, ...,   2.56,   2.56,   2.56]],
      dtype=float32), 'start': Period('2024-01-29', 'D'), 'feat_static_cat': array([0], dtyp

In [470]:
# Check the type and structure of 'forecast'
print(type(forecast))

<class 'gluonts.model.forecast.SampleForecast'>


In [471]:
print("Inp keys:", inp.keys())
print("Label keys:", label.keys())
print("Forecast attributes:", dir(forecast))
# If 'samples' is the key holding the forecast data
forecast_samples = forecast.samples
print("Shape of Forecast:", forecast_samples.shape)

Inp keys: dict_keys(['target', 'start', 'feat_static_cat'])
Label keys: dict_keys(['target', 'start', 'feat_static_cat'])
Forecast attributes: ['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs_ex__', '__gt__', '__hash__', '__init__', '__init_args__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_dim', '_index', '_mean', '_sorted_samples', '_sorted_samples_value', 'copy_aggregate', 'copy_dim', 'dim', 'freq', 'index', 'info', 'item_id', 'mean', 'mean_ts', 'median', 'num_samples', 'plot', 'prediction_length', 'quantile', 'quantile_ts', 'samples', 'start_date', 'to_quantile_forecast']
Shape of Forecast: (211, 28, 37)


In [472]:
# Extract the 'samples' attribute from the forecast object
forecast_samples = forecast.samples

# Convert the samples array into a DataFrame
forecast_df = pd.DataFrame(forecast_samples.reshape(-1, forecast_samples.shape[-1]))


# Display the first few rows of the DataFrame
print("Forecast DataFrame:\n", forecast_df)


Forecast DataFrame:
             0         1         2         3         4         5         6   \
0     6.244546  8.206137  8.157405  3.405497  1.041505  7.282986  3.689200   
1     6.056797  8.207906  8.027050  0.991974  0.004863  7.303753  3.597207   
2     6.238999  8.199000  8.006444  3.198909  1.514557  7.374465  3.969161   
3     6.297678  8.049164  8.142800  4.463728 -0.025934  7.264370  3.956424   
4     6.248057  8.390552  8.427096  3.417120  0.150546  7.616568  4.196241   
...        ...       ...       ...       ...       ...       ...       ...   
5903  5.893741  8.046805  7.836174  3.506464  0.081408  7.272997  3.574479   
5904  6.142601  8.138885  8.142800  3.506464  1.041505  7.084220  4.092190   
5905  6.509243  7.831586  8.284948  0.955661 -0.008814  7.264370  3.915500   
5906  6.647015  8.015854  8.324468  6.378257  0.000823  7.089613  3.929435   
5907  6.307086  8.218395  8.314522  0.159590  1.514557  7.211971  4.555245   

            7         8         9   ...   

In [473]:
mean_forecast = forecast_samples.mean(axis=1)  # Shape: (211, 37)
mean_forecast

array([[  6.197403 ,   8.253398 ,   8.139318 , ..., 309.68576  ,
          2.8009856,   2.9106944],
       [  6.2758975,   8.130506 ,   8.163931 , ..., 313.4554   ,
          2.7986166,   2.9631665],
       [  6.106342 ,   8.258499 ,   8.094522 , ..., 370.74188  ,
          2.8941553,   3.0208125],
       ...,
       [  6.2198715,   8.208741 ,   8.143265 , ..., 265.18097  ,
          2.946468 ,   2.950637 ],
       [  6.1843023,   8.109954 ,   8.120015 , ..., 286.27402  ,
          2.8735387,   3.0370047],
       [  6.2029963,   8.166093 ,   8.18982  , ..., 322.6876   ,
          2.9253485,   2.9455128]], dtype=float32)

In [474]:
# Extract the 'samples' attribute from the forecast object
forecast_samples = mean_forecast

# Convert the samples array into a DataFrame
forecast_df = pd.DataFrame(forecast_samples.reshape(-1, forecast_samples.shape[-1]))

# Display the first few rows of the DataFrame
print("Forecast DataFrame:\n", forecast_df)

Forecast DataFrame:
            0         1         2         3         4         5         6   \
0    6.197403  8.253398  8.139318  3.161876  0.345673  7.269594  3.792487   
1    6.275898  8.130506  8.163931  2.944256  0.305590  7.317292  3.846113   
2    6.106342  8.258499  8.094522  3.763843  0.762999  7.201531  3.872929   
3    6.239551  8.195918  8.104906  3.442468  7.232079  7.185257  3.832291   
4    6.220164  8.194661  8.120376  2.517165  0.537454  7.224599  3.886776   
..        ...       ...       ...       ...       ...       ...       ...   
206  6.184926  8.232120  8.099276  2.582122  0.341646  7.223845  3.831940   
207  6.218357  8.171310  8.149766  3.147372  1.092194  7.269664  3.870447   
208  6.219872  8.208741  8.143265  2.653437  1.117272  7.192854  3.889385   
209  6.184302  8.109954  8.120015  3.071867  7.420437  7.270527  3.793574   
210  6.202996  8.166093  8.189820  3.277520  3.205376  7.266912  3.868724   

           7         8         9   ...          27    

In [475]:
forecast_df = pd.DataFrame(forecast_samples)
forecast_df 

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,6.197403,8.253398,8.139318,3.161876,0.345673,7.269594,3.792487,2.575932,7.047272,7.416260,...,319.776917,8.377812,0.032863,8.310122,149.711685,187.061417,165.550919,309.685760,2.800986,2.910694
1,6.275898,8.130506,8.163931,2.944256,0.305590,7.317292,3.846113,2.930315,1.092738,7.486301,...,97.675797,8.442659,3.516186,8.324576,165.445831,176.744598,117.248047,313.455414,2.798617,2.963166
2,6.106342,8.258499,8.094522,3.763843,0.762999,7.201531,3.872929,3.974950,0.441106,7.461682,...,220.438431,8.807778,0.231792,8.222650,183.450287,192.122528,123.618156,370.741882,2.894155,3.020813
3,6.239551,8.195918,8.104906,3.442468,7.232079,7.185257,3.832291,2.923683,1.787995,7.452098,...,187.107910,8.511490,0.280100,8.300054,147.509933,183.681076,127.403816,365.819733,2.862999,3.033009
4,6.220164,8.194661,8.120376,2.517165,0.537454,7.224599,3.886776,2.570096,1.096733,7.484048,...,134.038651,8.524448,0.045339,8.241647,141.116470,204.045929,126.940781,410.417694,2.827590,2.973881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,6.184926,8.232120,8.099276,2.582122,0.341646,7.223845,3.831940,2.739332,1.541173,7.496598,...,70.101204,8.412252,1.057606,8.339730,144.265945,206.759186,132.867035,297.469757,2.946790,3.002488
207,6.218357,8.171310,8.149766,3.147372,1.092194,7.269664,3.870447,2.298895,0.506979,7.385982,...,175.899506,8.496291,7.049611,8.357068,161.782700,186.574081,130.987595,319.282501,2.948872,2.877228
208,6.219872,8.208741,8.143265,2.653437,1.117272,7.192854,3.889385,2.848049,2.580156,7.537475,...,165.941299,8.525853,0.475930,8.229085,124.227592,208.103653,141.184402,265.180969,2.946468,2.950637
209,6.184302,8.109954,8.120015,3.071867,7.420437,7.270527,3.793574,2.587708,1.274148,7.458023,...,247.703415,8.368110,0.010416,8.303389,162.516144,222.179474,142.146378,286.274017,2.873539,3.037005


In [476]:
# List of new column names
new_column_names = ['G620_water_level', 'NESRS1', 'NESRS2', 'NP205_PET', 'NP205_rain', 'NP205_stage', 
                    'NP62_stage', 'P33_PET', 'P33_rain', 'P33_stage', 'S12A', 'S12A_T', 'S12B', 'S12B_T', 
                    'S12C', 'S12C_T', 'S12D', 'S12D_T', 'S18C_S', 'S199', 'S200', 'S332B', 'S332BN', 'S332C', 
                    'S332D', 'S332DX1', 'S333', 'S333N', 'S333_T', 'S334', 'S334_H', 'S343A', 'S343B', 'S344', 
                    'S356', 'SWEVER4_stage', 'TSH_stage']

# Renaming the columns dynamically
forecast_df.columns = new_column_names

# Verifying the change
print(forecast_df.columns)

Index(['G620_water_level', 'NESRS1', 'NESRS2', 'NP205_PET', 'NP205_rain',
       'NP205_stage', 'NP62_stage', 'P33_PET', 'P33_rain', 'P33_stage', 'S12A',
       'S12A_T', 'S12B', 'S12B_T', 'S12C', 'S12C_T', 'S12D', 'S12D_T',
       'S18C_S', 'S199', 'S200', 'S332B', 'S332BN', 'S332C', 'S332D',
       'S332DX1', 'S333', 'S333N', 'S333_T', 'S334', 'S334_H', 'S343A',
       'S343B', 'S344', 'S356', 'SWEVER4_stage', 'TSH_stage'],
      dtype='object')


In [477]:
forecast_df

,G620_water_level,NESRS1,NESRS2,NP205_PET,NP205_rain,NP205_stage,NP62_stage,P33_PET,P33_rain,P33_stage,...,S333N,S333_T,S334,S334_H,S343A,S343B,S344,S356,SWEVER4_stage,TSH_stage
0,6.197403,8.253398,8.139318,3.161876,0.345673,7.269594,3.792487,2.575932,7.047272,7.416260,...,319.776917,8.377812,0.032863,8.310122,149.711685,187.061417,165.550919,309.685760,2.800986,2.910694
1,6.275898,8.130506,8.163931,2.944256,0.305590,7.317292,3.846113,2.930315,1.092738,7.486301,...,97.675797,8.442659,3.516186,8.324576,165.445831,176.744598,117.248047,313.455414,2.798617,2.963166
2,6.106342,8.258499,8.094522,3.763843,0.762999,7.201531,3.872929,3.974950,0.441106,7.461682,...,220.438431,8.807778,0.231792,8.222650,183.450287,192.122528,123.618156,370.741882,2.894155,3.020813
3,6.239551,8.195918,8.104906,3.442468,7.232079,7.185257,3.832291,2.923683,1.787995,7.452098,...,187.107910,8.511490,0.280100,8.300054,147.509933,183.681076,127.403816,365.819733,2.862999,3.033009
4,6.220164,8.194661,8.120376,2.517165,0.537454,7.224599,3.886776,2.570096,1.096733,7.484048,...,134.038651,8.524448,0.045339,8.241647,141.116470,204.045929,126.940781,410.417694,2.827590,2.973881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,6.184926,8.232120,8.099276,2.582122,0.341646,7.223845,3.831940,2.739332,1.541173,7.496598,...,70.101204,8.412252,1.057606,8.339730,144.265945,206.759186,132.867035,297.469757,2.946790,3.002488
207,6.218357,8.171310,8.149766,3.147372,1.092194,7.269664,3.870447,2.298895,0.506979,7.385982,...,175.899506,8.496291,7.049611,8.357068,161.782700,186.574081,130.987595,319.282501,2.948872,2.877228
208,6.219872,8.208741,8.143265,2.653437,1.117272,7.192854,3.889385,2.848049,2.580156,7.537475,...,165.941299,8.525853,0.475930,8.229085,124.227592,208.103653,141.184402,265.180969,2.946468,2.950637
209,6.184302,8.109954,8.120015,3.071867,7.420437,7.270527,3.793574,2.587708,1.274148,7.458023,...,247.703415,8.368110,0.010416,8.303389,162.516144,222.179474,142.146378,286.274017,2.873539,3.037005


In [478]:
# Forecast_df is already in wide format with unique_id as columns

# 1. Convert Y_test_df to wide format
Y_test_wide = Y_test_df.pivot(index='ds', columns='unique_id', values='y')

# 2. Now, Y_test_wide is in wide format, where each unique_id is a column
# Merge or align it with forecast_df
# You can reset the index if you need to work with it as a regular DataFrame
Y_test_wide.reset_index(inplace=True)
Y_test_wide

unique_id,ds,G620_water_level,NESRS1,NESRS2,NP205_PET,NP205_rain,NP205_stage,NP62_stage,P33_PET,P33_rain,...,S333N,S333_T,S334,S334_H,S343A,S343B,S344,S356,SWEVER4_stage,TSH_stage
0,2024-01-29,5.99,7.99,7.99,4.09,0.00,7.127,3.69,4.09,0.00,...,238.84,8.48,0.0,8.23,159.32,216.57,-14.86,394.49,2.70,2.70
1,2024-01-30,5.97,7.98,7.98,3.72,0.00,7.116,3.68,3.72,0.00,...,215.67,8.48,0.0,8.20,159.04,214.94,-43.99,393.54,2.68,2.69
2,2024-01-31,5.96,7.97,7.98,3.98,0.00,7.105,3.67,3.98,0.00,...,219.56,8.28,0.0,8.20,157.15,212.66,-68.76,393.81,2.66,2.67
3,2024-02-01,5.95,7.97,7.97,3.14,0.00,7.096,3.65,3.14,0.00,...,226.68,8.58,0.0,8.19,155.52,211.21,-50.62,394.94,2.64,2.66
4,2024-02-02,5.94,7.96,7.97,3.40,0.00,7.090,3.64,3.40,0.00,...,249.01,8.47,0.0,8.19,155.34,210.66,-21.54,399.75,2.63,2.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2024-08-22,6.13,8.20,8.18,2.75,1.72,7.289,3.88,2.75,0.05,...,80.18,8.29,0.0,8.37,137.74,239.14,142.83,464.05,2.85,2.78
207,2024-08-23,6.18,8.20,8.18,2.75,0.04,7.360,3.88,2.75,0.00,...,73.23,8.00,0.0,8.38,157.46,250.11,142.83,527.70,2.90,2.81
208,2024-08-24,6.20,8.21,8.20,2.75,0.00,7.360,3.88,2.75,0.01,...,75.46,8.47,0.0,8.37,154.59,251.30,142.83,524.43,2.88,2.80
209,2024-08-25,6.19,8.20,8.18,2.75,0.00,7.345,3.98,2.75,0.27,...,126.09,8.27,0.0,8.38,157.40,251.77,142.83,525.73,2.86,2.80


In [479]:
# 1. Pivot the Y_test_df to wide format
Y_test_wide = Y_test_df.pivot(index='ds', columns='unique_id', values='y')

# 2. Reset the index so that 'ds' becomes a regular column
Y_test_wide.reset_index(inplace=True)

# 3. If you want to drop the original 'unique_id' index, you can just reassign the columns
Y_test_wide.columns.name = None  # Remove the columns' name, which is 'unique_id'
Y_test_wide.reset_index(drop=True, inplace=True)  # Drop the 'index' column if it was added

# Verify the result
print(Y_test_wide.head())

          ds  G620_water_level  NESRS1  NESRS2  NP205_PET  NP205_rain  \
0 2024-01-29              5.99    7.99    7.99       4.09         0.0   
1 2024-01-30              5.97    7.98    7.98       3.72         0.0   
2 2024-01-31              5.96    7.97    7.98       3.98         0.0   
3 2024-02-01              5.95    7.97    7.97       3.14         0.0   
4 2024-02-02              5.94    7.96    7.97       3.40         0.0   

   NP205_stage  NP62_stage  P33_PET  P33_rain  ...   S333N  S333_T  S334  \
0        7.127        3.69     4.09       0.0  ...  238.84    8.48   0.0   
1        7.116        3.68     3.72       0.0  ...  215.67    8.48   0.0   
2        7.105        3.67     3.98       0.0  ...  219.56    8.28   0.0   
3        7.096        3.65     3.14       0.0  ...  226.68    8.58   0.0   
4        7.090        3.64     3.40       0.0  ...  249.01    8.47   0.0   

   S334_H   S343A   S343B   S344    S356  SWEVER4_stage  TSH_stage  
0    8.23  159.32  216.57 -14.86  3

In [480]:
Y_test_wide

,ds,G620_water_level,NESRS1,NESRS2,NP205_PET,NP205_rain,NP205_stage,NP62_stage,P33_PET,P33_rain,...,S333N,S333_T,S334,S334_H,S343A,S343B,S344,S356,SWEVER4_stage,TSH_stage
0,2024-01-29,5.99,7.99,7.99,4.09,0.00,7.127,3.69,4.09,0.00,...,238.84,8.48,0.0,8.23,159.32,216.57,-14.86,394.49,2.70,2.70
1,2024-01-30,5.97,7.98,7.98,3.72,0.00,7.116,3.68,3.72,0.00,...,215.67,8.48,0.0,8.20,159.04,214.94,-43.99,393.54,2.68,2.69
2,2024-01-31,5.96,7.97,7.98,3.98,0.00,7.105,3.67,3.98,0.00,...,219.56,8.28,0.0,8.20,157.15,212.66,-68.76,393.81,2.66,2.67
3,2024-02-01,5.95,7.97,7.97,3.14,0.00,7.096,3.65,3.14,0.00,...,226.68,8.58,0.0,8.19,155.52,211.21,-50.62,394.94,2.64,2.66
4,2024-02-02,5.94,7.96,7.97,3.40,0.00,7.090,3.64,3.40,0.00,...,249.01,8.47,0.0,8.19,155.34,210.66,-21.54,399.75,2.63,2.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2024-08-22,6.13,8.20,8.18,2.75,1.72,7.289,3.88,2.75,0.05,...,80.18,8.29,0.0,8.37,137.74,239.14,142.83,464.05,2.85,2.78
207,2024-08-23,6.18,8.20,8.18,2.75,0.04,7.360,3.88,2.75,0.00,...,73.23,8.00,0.0,8.38,157.46,250.11,142.83,527.70,2.90,2.81
208,2024-08-24,6.20,8.21,8.20,2.75,0.00,7.360,3.88,2.75,0.01,...,75.46,8.47,0.0,8.37,154.59,251.30,142.83,524.43,2.88,2.80
209,2024-08-25,6.19,8.20,8.18,2.75,0.00,7.345,3.98,2.75,0.27,...,126.09,8.27,0.0,8.38,157.40,251.77,142.83,525.73,2.86,2.80


In [481]:
# Ensure both DataFrames have matching indices
forecast_df['ds'] = Y_test_wide['ds'].values


In [482]:
forecast_df


,G620_water_level,NESRS1,NESRS2,NP205_PET,NP205_rain,NP205_stage,NP62_stage,P33_PET,P33_rain,P33_stage,...,S333_T,S334,S334_H,S343A,S343B,S344,S356,SWEVER4_stage,TSH_stage,ds
0,6.197403,8.253398,8.139318,3.161876,0.345673,7.269594,3.792487,2.575932,7.047272,7.416260,...,8.377812,0.032863,8.310122,149.711685,187.061417,165.550919,309.685760,2.800986,2.910694,2024-01-29
1,6.275898,8.130506,8.163931,2.944256,0.305590,7.317292,3.846113,2.930315,1.092738,7.486301,...,8.442659,3.516186,8.324576,165.445831,176.744598,117.248047,313.455414,2.798617,2.963166,2024-01-30
2,6.106342,8.258499,8.094522,3.763843,0.762999,7.201531,3.872929,3.974950,0.441106,7.461682,...,8.807778,0.231792,8.222650,183.450287,192.122528,123.618156,370.741882,2.894155,3.020813,2024-01-31
3,6.239551,8.195918,8.104906,3.442468,7.232079,7.185257,3.832291,2.923683,1.787995,7.452098,...,8.511490,0.280100,8.300054,147.509933,183.681076,127.403816,365.819733,2.862999,3.033009,2024-02-01
4,6.220164,8.194661,8.120376,2.517165,0.537454,7.224599,3.886776,2.570096,1.096733,7.484048,...,8.524448,0.045339,8.241647,141.116470,204.045929,126.940781,410.417694,2.827590,2.973881,2024-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,6.184926,8.232120,8.099276,2.582122,0.341646,7.223845,3.831940,2.739332,1.541173,7.496598,...,8.412252,1.057606,8.339730,144.265945,206.759186,132.867035,297.469757,2.946790,3.002488,2024-08-22
207,6.218357,8.171310,8.149766,3.147372,1.092194,7.269664,3.870447,2.298895,0.506979,7.385982,...,8.496291,7.049611,8.357068,161.782700,186.574081,130.987595,319.282501,2.948872,2.877228,2024-08-23
208,6.219872,8.208741,8.143265,2.653437,1.117272,7.192854,3.889385,2.848049,2.580156,7.537475,...,8.525853,0.475930,8.229085,124.227592,208.103653,141.184402,265.180969,2.946468,2.950637,2024-08-24
209,6.184302,8.109954,8.120015,3.071867,7.420437,7.270527,3.793574,2.587708,1.274148,7.458023,...,8.368110,0.010416,8.303389,162.516144,222.179474,142.146378,286.274017,2.873539,3.037005,2024-08-25


In [483]:
#Convert 'date' to datetime format if not 
forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])

#Melt the DataFrame to long format
long_df = pd.melt(forecast_df, id_vars=['ds'], 
                  value_vars=[col for col in forecast_df.columns if col != 'ds'], 
                  var_name='unique_id', 
                  value_name='y')

# Display the resulting long format DataFrame
print(long_df) 

             ds         unique_id         y
0    2024-01-29  G620_water_level  6.197403
1    2024-01-30  G620_water_level  6.275898
2    2024-01-31  G620_water_level  6.106342
3    2024-02-01  G620_water_level  6.239551
4    2024-02-02  G620_water_level  6.220164
...         ...               ...       ...
7802 2024-08-22         TSH_stage  3.002488
7803 2024-08-23         TSH_stage  2.877228
7804 2024-08-24         TSH_stage  2.950637
7805 2024-08-25         TSH_stage  3.037005
7806 2024-08-26         TSH_stage  2.945513

[7807 rows x 3 columns]


In [484]:
#Rename the date column to 'ds'
long_df.rename(columns={'y': 'Morai28'}, inplace=True)
long_df

,ds,unique_id,Morai28
0,2024-01-29,G620_water_level,6.197403
1,2024-01-30,G620_water_level,6.275898
2,2024-01-31,G620_water_level,6.106342
3,2024-02-01,G620_water_level,6.239551
4,2024-02-02,G620_water_level,6.220164
...,...,...,...
7802,2024-08-22,TSH_stage,3.002488
7803,2024-08-23,TSH_stage,2.877228
7804,2024-08-24,TSH_stage,2.950637
7805,2024-08-25,TSH_stage,3.037005


In [485]:
results_df = pd.merge(long_df, Y_test_df, on=['unique_id', 'ds'], how='inner')
results_df

,ds,unique_id,Morai28,y
0,2024-01-29,G620_water_level,6.197403,5.99
1,2024-01-30,G620_water_level,6.275898,5.97
2,2024-01-31,G620_water_level,6.106342,5.96
3,2024-02-01,G620_water_level,6.239551,5.95
4,2024-02-02,G620_water_level,6.220164,5.94
...,...,...,...,...
7802,2024-08-22,TSH_stage,3.002488,2.78
7803,2024-08-23,TSH_stage,2.877228,2.81
7804,2024-08-24,TSH_stage,2.950637,2.80
7805,2024-08-25,TSH_stage,3.037005,2.80


In [486]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# List of stages to evaluate
stages = ['NP205_stage', 'P33_stage','G620_water_level', 'NESRS1', 'NESRS2']

for stage in stages:
    # Filter the data for the specific stage
    stage_data = results_df[results_df['unique_id'] == stage]
    print(f"======== {stage} ========")
    
    # Extract the actual and predicted values
    true = stage_data['y'].values
    pred = stage_data['Morai28'].values
    
    # Calculate MAE and RMSE
    mae_value = mae(true, pred)
    rmse_value = np.sqrt(mse(true, pred))
    
    # Print the results
    print(f'MAE = {mae_value:.3f}')
    print(f'RMSE = {rmse_value:.3f}\n')

======== NP205_stage ========
MAE = 0.555
RMSE = 0.820

======== P33_stage ========
MAE = 0.331
RMSE = 0.422

======== G620_water_level ========
MAE = 0.446
RMSE = 0.569

======== NESRS1 ========
MAE = 0.261
RMSE = 0.351

======== NESRS2 ========
MAE = 0.224
RMSE = 0.305



In [487]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# List of stages to evaluate
stages = ['NP205_stage', 'P33_stage', 'G620_water_level', 'NESRS1', 'NESRS2']

# List of models (these correspond to column names in the DataFrame)
models = ['Morai28']

# Dictionary to store overall MAE and RMSE for each model
overall_mae = {model: [] for model in models}
overall_rmse = {model: [] for model in models}

# Iterate through each stage
for stage in stages:
    # Filter the data for the specific stage
    stage_data = results_df[results_df['unique_id'] == stage]
    
    # Check if data for the stage exists
    if stage_data.empty:
        print(f"No data found for {stage}. Skipping...\n")
        continue
    
    # Extract the actual values
    true = stage_data['y'].values
    
    # Iterate through each model
    for model in models:
        # Extract the predicted values for the model
        pred = stage_data[model].values
        
        # Calculate MAE and RMSE, then append to the corresponding model's list
        model_mae = mae(true, pred)
        model_rmse = np.sqrt(mse(true, pred))
        
        overall_mae[model].append(model_mae)
        overall_rmse[model].append(model_rmse)

# Calculate the overall MAE and RMSE (mean of MAEs and RMSEs across the 5 stations) for each model
final_overall_mae = {model: np.mean(maes) for model, maes in overall_mae.items()}
final_overall_rmse = {model: np.mean(rmses) for model, rmses in overall_rmse.items()}

# Print the overall MAE and RMSE for each model
print("Overall MAE for Morai28(only for selected stages):")
for model, mae_value in final_overall_mae.items():
    print(f"{model}: MAE = {mae_value:.3f}")

print("\nOverall RMSE for Morai28(only for selected stages):")
for model, rmse_value in final_overall_rmse.items():
    print(f"{model}: RMSE = {rmse_value:.3f}")

Overall MAE for Morai28(only for selected stages):
Morai28: MAE = 0.364

Overall RMSE for Morai28(only for selected stages):
Morai28: RMSE = 0.493


In [ ]:
# Save predictions to a CSV file
results_df.to_csv('Output/Morai-Predictions-28days.csv', index=False)